In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )


intitate_notebook()

Current directory for notebook:  /workspace


In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PostgresETL")
    .config("spark.jars", "setup_files/postgresql-42.7.5.jar")
    .getOrCreate()
)


25/04/20 12:04:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
username = "data_source_user"
password = "data_source_user_password"
host = "172.17.0.1"
port = "5435"
database = "data_source_db"


jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}"
properties = {"user": username, "password": password, "driver": "org.postgresql.Driver"}

In [8]:
df = spark.read.jdbc(url=jdbc_url, table="application_train", properties=properties)
df

DataFrame[SK_ID_CURR: bigint, TARGET: bigint, NAME_CONTRACT_TYPE: string, CODE_GENDER: string, FLAG_OWN_CAR: string, FLAG_OWN_REALTY: string, CNT_CHILDREN: bigint, AMT_INCOME_TOTAL: double, AMT_CREDIT: double, AMT_ANNUITY: double, AMT_GOODS_PRICE: double, NAME_TYPE_SUITE: string, NAME_INCOME_TYPE: string, NAME_EDUCATION_TYPE: string, NAME_FAMILY_STATUS: string, NAME_HOUSING_TYPE: string, REGION_POPULATION_RELATIVE: double, DAYS_BIRTH: bigint, DAYS_EMPLOYED: bigint, DAYS_REGISTRATION: double, DAYS_ID_PUBLISH: bigint, OWN_CAR_AGE: double, FLAG_MOBIL: bigint, FLAG_EMP_PHONE: bigint, FLAG_WORK_PHONE: bigint, FLAG_CONT_MOBILE: bigint, FLAG_PHONE: bigint, FLAG_EMAIL: bigint, OCCUPATION_TYPE: string, CNT_FAM_MEMBERS: double, REGION_RATING_CLIENT: bigint, REGION_RATING_CLIENT_W_CITY: bigint, WEEKDAY_APPR_PROCESS_START: string, HOUR_APPR_PROCESS_START: bigint, REG_REGION_NOT_LIVE_REGION: bigint, REG_REGION_NOT_WORK_REGION: bigint, LIVE_REGION_NOT_WORK_REGION: bigint, REG_CITY_NOT_LIVE_CITY: big

In [7]:
from src.data.explore_column_info import get_column_summary

In [9]:
for col in df.columns:
    get_column_summary(df=df, column_name=col)
    print("\n\n")

Summary for column: SK_ID_CURR
Data Type        : LongType()
Unique Values    : 307511
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Top 5 unique values : [100031, 100501, 100565, 100847, 100852]



Summary for column: TARGET
Data Type        : LongType()
Unique Values    : 2
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Top 5 unique values : [0, 1]



Summary for column: NAME_CONTRACT_TYPE
Data Type        : StringType()
Unique Values    : 2
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Top 5 unique values : ['Cash loans', 'Revolving loans']



Summary for column: CODE_GENDER
Data Type        : StringType()
Unique Values    : 3
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Top 5 unique values : ['F', 'M', 'XNA']



Summary for column: FLAG_OWN_CAR
Data Type        : StringType()
Unique Values    : 2
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Top 5 unique

Summary for column: EXT_SOURCE_3
Data Type        : DoubleType()
Unique Values    : 814
Null Count       : 60965
Total Rows       : 307511
Null Percentage  : 19.825307062186393 %
Top 5 unique values : [0.746300213050371, 0.7136313997323308, 0.6940926425266661, 0.6706517530862718, 0.6528965519806539]





Summary for column: APARTMENTS_AVG
Data Type        : DoubleType()
Unique Values    : 2339
Null Count       : 156061
Total Rows       : 307511
Null Percentage  : 50.74972927797705 %
Top 5 unique values : [0.0825, 0.0619, 0.0928, 0.0722, 0.0082]



Summary for column: BASEMENTAREA_AVG
Data Type        : DoubleType()
Unique Values    : 3780
Null Count       : 179943
Total Rows       : 307511
Null Percentage  : 58.51595552679416 %
Top 5 unique values : [0.0, 0.0545, 0.0818, 0.0727, 0.1091]



Summary for column: YEARS_BEGINEXPLUATATION_AVG
Data Type        : DoubleType()
Unique Values    : 285
Null Count       : 150007
Total Rows       : 307511
Null Percentage  : 48.781019215572776 %
Top 5 unique values : [0.9871, 0.9856, 0.9861, 0.9801, 0.9866]



Summary for column: YEARS_BUILD_AVG
Data Type        : DoubleType()
Unique Values    : 149
Null Count       : 204488
Total Rows       : 307511
Null Percentage  : 66.49778381911543 %
Top 5 unique values : [0.8232, 0.8164, 0.8028, 0.728, 0.7348]


Summary for column: ENTRANCES_AVG
Data Type        : DoubleType()
Unique Values    : 285
Null Count       : 154828
Total Rows       : 307511
Null Percentage  : 50.3487680115508 %


Top 5 unique values : [0.1379, 0.069, 0.1034, 0.2069, 0.0345]



Summary for column: FLOORSMAX_AVG
Data Type        : DoubleType()
Unique Values    : 403
Null Count       : 153020
Total Rows       : 307511
Null Percentage  : 49.76082156410665 %
Top 5 unique values : [0.1667, 0.3333, 0.0417, 0.375, 0.125]



Summary for column: FLOORSMIN_AVG
Data Type        : DoubleType()
Unique Values    : 305
Null Count       : 208642
Total Rows       : 307511
Null Percentage  : 67.84862980511267 %
Top 5 unique values : [0.2083, 0.375, 0.0417, 0.0833, 0.4167]



Summary for column: LANDAREA_AVG
Data Type        : DoubleType()
Unique Values    : 3527
Null Count       : 182590
Total Rows       : 307511
Null Percentage  : 59.376737742714894 %
Top 5 unique values : [0.0, 0.0631, 0.0316, 0.0473, 0.0174]



Summary for column: LIVINGAPARTMENTS_AVG
Data Type        : DoubleType()
Unique Values    : 1868
Null Count       : 210199
Total Rows       : 307511
Null Percentage  : 68.35495315614726 %
Top 5 unique v

Summary for column: NONLIVINGAREA_AVG
Data Type        : DoubleType()
Unique Values    : 3290
Null Count       : 169682
Total Rows       : 307511
Null Percentage  : 55.17916432257708 %
Top 5 unique values : [0.0, 0.0012, 0.0044, 0.0022, 0.0031]





Summary for column: APARTMENTS_MODE
Data Type        : DoubleType()
Unique Values    : 760
Null Count       : 156061
Total Rows       : 307511
Null Percentage  : 50.74972927797705 %
Top 5 unique values : [0.084, 0.063, 0.0945, 0.0735, 0.0084]



Summary for column: BASEMENTAREA_MODE
Data Type        : DoubleType()
Unique Values    : 3841
Null Count       : 179943
Total Rows       : 307511
Null Percentage  : 58.51595552679416 %
Top 5 unique values : [0.0, 0.0566, 0.0849, 0.0642, 0.083]



Summary for column: YEARS_BEGINEXPLUATATION_MODE
Data Type        : DoubleType()
Unique Values    : 221
Null Count       : 150007
Total Rows       : 307511
Null Percentage  : 48.781019215572776 %
Top 5 unique values : [0.9871, 0.9866, 0.9861, 0.9801, 0.9806]



Summary for column: YEARS_BUILD_MODE
Data Type        : DoubleType()
Unique Values    : 154
Null Count       : 204488
Total Rows       : 307511
Null Percentage  : 66.49778381911543 %
Top 5 unique values : [0.8301, 0.8236, 0.7387, 0.8171, 0.8105]

Top 5 unique values : ['reg oper account', 'reg oper spec account', 'not specified', 'org spec account', None]



Summary for column: HOUSETYPE_MODE
Data Type        : StringType()
Unique Values    : 3
Null Count       : 154297
Total Rows       : 307511
Null Percentage  : 50.176091261776 %
Top 5 unique values : ['block of flats', 'specific housing', 'terraced house', None]



Summary for column: TOTALAREA_MODE
Data Type        : DoubleType()
Unique Values    : 5116
Null Count       : 148431
Total Rows       : 307511
Null Percentage  : 48.26851722377411 %
Top 5 unique values : [0.0, 0.057, 0.0547, 0.055, 0.0548]



Summary for column: WALLSMATERIAL_MODE
Data Type        : StringType()
Unique Values    : 7
Null Count       : 156341
Total Rows       : 307511
Null Percentage  : 50.8407829313423 %
Top 5 unique values : ['Panel', 'Stone, brick', 'Block', 'Wooden', 'Mixed']



Summary for column: EMERGENCYSTATE_MODE
Data Type        : StringType()
Unique Values    : 2
Null Count       : 14575

Summary for column: OBS_60_CNT_SOCIAL_CIRCLE
Data Type        : DoubleType()
Unique Values    : 33
Null Count       : 1021
Total Rows       : 307511
Null Percentage  : 0.3320206431639844 %


Top 5 unique values : [0.0, 1.0, 2.0, 3.0, 4.0]



Summary for column: DEF_60_CNT_SOCIAL_CIRCLE
Data Type        : DoubleType()
Unique Values    : 9
Null Count       : 1021
Total Rows       : 307511
Null Percentage  : 0.3320206431639844 %
Top 5 unique values : [0.0, 1.0, 2.0, 3.0, 4.0]



Summary for column: DAYS_LAST_PHONE_CHANGE
Data Type        : DoubleType()
Unique Values    : 3773
Null Count       : 1
Total Rows       : 307511
Null Percentage  : 0.00032519161916159093 %
Top 5 unique values : [0.0, -1.0, -2.0, -3.0, -4.0]



Summary for column: FLAG_DOCUMENT_2
Data Type        : LongType()
Unique Values    : 2
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Top 5 unique values : [0, 1]



Summary for column: FLAG_DOCUMENT_3
Data Type        : LongType()
Unique Values    : 2
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Top 5 unique values : [1, 0]



Summary for column: FLAG_DOCUMENT_4
Data Type        : LongType()
Unique Values    